# Monitoring a Model

When you've deployed a model into productipon as a service, you'll want to monitor it to track usage and explore the requests it processes. In this lab, you'll use Azure Application Insights to monitor activity for a model service endpoint.


## Connect to Your Workspace

The first thing you need to do is to connect to your workspace using the Azure ML SDK.

> **Note**: If the authenticated session with your Azure subscription has expired since you completed the previous exercise, you'll be prompted to reauthenticate.

In [ ]:
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

## Create an AKS Compute target

You'll need to create an Azure Kubetrnetes Service (AKS) cluster to host the service. Run the following cell to do this.

> **Note**: This can take some time!

In [ ]:
from azureml.core.compute import ComputeTarget, AksCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = 'aks-compute'

try:
    # get the cluster if it exists
    production_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If not, create it
    compute_config = AksCompute.provisioning_configuration(location='eastus')
    production_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

production_cluster.wait_for_completion(show_output=True)
print('Cluster ready.')

## Prepare a Model for Deployment

Now we need a model to deploy. Run the code below to:

1. Create and register a dataset.
2. Train a model using the dataset.
3. Register the model.

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from azureml.core import Dataset

# Upload data files to the default datastore
default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'],
                       target_path='diabetes-data/',
                       overwrite=True,
                       show_progress=True)

#Create a tabular dataset from the path on the datastore
data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# Register the tabular dataset
data_set = data_set.register(workspace=ws, 
                           name='diabetes dataset',
                           description='diabetes data',
                           tags = {'format':'CSV'},
                           create_new_version=True)


# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = "diabetes-training")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
diabetes = data_set.to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']},
                   datasets=[('Training Data', data_set)])

# Get the registered model
model = ws.models['diabetes_model']

print('Model trained and registered.')

## Deploy a Model as a Web Service

Now you're ready to deploy the registered model as a web service.

First, create a folder for the deployment configuration files

In [ ]:
import os

folder_name = 'diabetes_service'
os.makedirs(folder_name, exist_ok=True)
print(folder_name)

Now you need an entry script that the service will use to score new data.

In [ ]:
%%writefile $folder_name/score_diabetes.py
import json
import numpy as np
import os
import joblib
from azureml.core.model import Model
#import azureml.train.automl # Required for AutoML models with pre-processing

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data - the features of patients to be classified.
    data = json.loads(raw_data)['data']
    # Get a prediction from the model
    predictions = model.predict(data)
    # print the data and predictions (so they'll be logged!)
    print('Data:' + data + 'Predictions:' + predictions)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

You'll also need a Conda configuration file for the service environment.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
#myenv.add_pip_package("azureml-sdk[automl]") # Required for AutoML models

# Save the environment config as a .yml file
env_file = folder_name + "/diabetes_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

Now you can deploy the service to the AKS cluster.

In [ ]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = folder_name,
                                   entry_script="score_diabetes.py",
                                   conda_file="diabetes_env.yml")

aks_target = AksCompute(ws,'aks-compute')

service_name = "diabetes-service-aks"
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
aks_service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

## Enable Application Insights

Next, you need to enable Application Insights for the service.

In [ ]:
# Enable AppInsights
aks_service.update(enable_app_insights=True)
print('AppInsights enabled!')

## Use the Web Service

With the service deployed, now you can consume it from a client application.

First, determine the URL to which these applications must submit their requests, and the authorization key they need to provide.

In [ ]:
endpoint = aks_service.scoring_uri
print(endpoint)
primary, secondary = aks_service.get_keys()
print(primary)

Now that you know the endpoint URI, an application can simply make an HTTP request, sending the patient data in JSON (or binary) format, and receive back the predicted class(es).

In [ ]:
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }
headers['Authorization'] = f'Bearer {primary}'

predictions = requests.post(endpoint, input_json, headers = headers)
print(predictions.status_code)
if predictions.status_code == 200:
    predicted_classes = json.loads(predictions.json())
    for i in range(len(x_new)):
        print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Now you can view the data logged for the service endpoint:
1. In the [Azure portal](https://portal.azure.com), open your Machine Learning workspace.
2. On the **Overview** page, click the link for the associated **Application Insights** resource.
3. On the Application Insights blade, click **Logs (Analytics)**. Then click **Get Started**. 
4. If a tip explaining how rto write a query is displayed, close it.
5. Paste the following query into the query editor and click **Run**
    ```
    traces
    |where  message == "STDOUT"
    |project  timestamp, customDimensions.Content
    ```
6. View the results, noting the data and predictions that were logged.

If you don't intend to complete the next lab, run the cell below to delete the service and AKS cluster to minimize cost.

In [ ]:
aks_service.delete()
print('Service deleted!')
production_cluster.delete()
print('Cluster deletion requested!')